# CUSTOMER LIFETIME VALUE (Müşteri Yaşam Boyu Değeri)


### Veri Seti Hikayesi

Online Retail II isimli veri seti İngiltere merkezli online bir satış mağazasının 01/12/2009 - 09/12/2011 tarihleri arasındaki satışlarını içeriyor.
Kaynak:https://archive.ics.uci.edu/ml/datasets/Online+Retail+II

 Değişkenler
 InvoiceNo: Fatura numarası. Her işleme yani faturaya ait eşsiz numara. C ile başlıyorsa iptal edilen işlem.
 
 StockCode: Ürün kodu. Her bir ürün için eşsiz numara.
 Description: Ürün ismi
 
 Quantity: Ürün adedi. Faturalardaki ürünlerden kaçar tane satıldığını ifade etmektedir.
 
 InvoiceDate: Fatura tarihi ve zamanı.
 
 UnitPrice: Ürün fiyatı (Sterlin cinsinden)
 
 CustomerID: Eşsiz müşteri numarası
 
 Country: Ülke ismi. Müşterinin yaşadığı ülke.

-----------

## 1. Veri Hazırlama

In [2]:
#kütüphanelerin import edilmesi 
import pandas as pd 
from sklearn.preprocessing import MinMaxScaler

pd.set_option("display.max.columns",None)
# pd.set_option("display.max_rows",None)
pd.set_option('display.float_format', lambda x: '%.5f' % x)

In [3]:
# datasetin import edilmesi

df_ =pd.read_excel(r"C:\Users\kkakt\Desktop\CRM\datasets\online_retail_II.xlsx",sheet_name="Year 2009-2010")
df= df_.copy()
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95000,13085.00000,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75000,13085.00000,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75000,13085.00000,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10000,13085.00000,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25000,13085.00000,United Kingdom


In [4]:
#iade işlemlerini tablodan çıkarmak 
print(df.shape[0])
df = df[~(df["Invoice"].str.contains("C",na=False))]
print(df.shape[0])

525461
515255


In [7]:
df = df[df["Quantity"]>0]

In [8]:
#eksik değerleri çıkarmak
df.dropna(inplace=True)
print(df.shape[0])

407695


In [9]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Quantity,407695.00000,13.58669,96.84223,1.00000,2.00000,5.00000,12.00000,19152.00000
Price,407695.00000,3.29419,34.75666,0.00000,1.25000,1.95000,3.75000,10953.50000
Customer ID,407695.00000,15368.50411,1679.79570,12346.00000,13997.00000,15321.00000,16812.00000,18287.00000


In [10]:
# toplam fiyat değişkeni oluşturma

df["total_price"] = df["Quantity"] * df["Price"]
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,total_price
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95000,13085.00000,United Kingdom,83.40000
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75000,13085.00000,United Kingdom,81.00000
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75000,13085.00000,United Kingdom,81.00000
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10000,13085.00000,United Kingdom,100.80000
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25000,13085.00000,United Kingdom,30.00000


In [19]:
# cltv için df hazırlama

cltv_c = df.groupby("Customer ID").agg({
    "Invoice" : lambda x:x.nunique(),
    "Quantity": lambda x:x.sum(),
    "total_price" : lambda x:x.sum()
})
cltv_c.columns = ["total_transaction","total_unit","total_price"]
cltv_c.head()

,total_transaction,total_unit,total_price
Customer ID,,,
12346.00000,11,70,372.86000
12347.00000,2,828,1323.32000
12348.00000,1,373,222.16000
12349.00000,3,993,2671.14000
12351.00000,1,261,300.93000


## Average Order Value

### average_order_value =  total_price / total_transaction

In [22]:
cltv_c["average_order_value"] = cltv_c["total_price"] / cltv_c["total_transaction"]
cltv_c.head()

,total_transaction,total_unit,total_price,average_order_value
Customer ID,,,,
12346.00000,11,70,372.86000,33.89636
12347.00000,2,828,1323.32000,661.66000
12348.00000,1,373,222.16000,222.16000
12349.00000,3,993,2671.14000,890.38000
12351.00000,1,261,300.93000,300.93000


## 3. Purchase Frequence 
### purchase_frequence = (total_transaction / total_number_of_custormer)

In [23]:
#toplam eşsiz müşteri
tnc = cltv_c.shape[0]
tnc

cltv_c["purchase_frequence"] = cltv_c["total_transaction"] / tnc
cltv_c.head()

,total_transaction,total_unit,total_price,average_order_value,purchase_frequence
Customer ID,,,,,
12346.00000,11,70,372.86000,33.89636,0.00255
12347.00000,2,828,1323.32000,661.66000,0.00046
12348.00000,1,373,222.16000,222.16000,0.00023
12349.00000,3,993,2671.14000,890.38000,0.00070
12351.00000,1,261,300.93000,300.93000,0.00023


## 4. Repeat Rate & Churn Rate 
### repeat_rate = (birden fazla alışveriş yapan müşteri sayısı / tüm müşteriler)
### churn_rate = 1-repeat_rate

In [27]:
#repeat_rate hesaplama
repeat_rate = cltv_c[cltv_c["total_transaction"]>1].shape[0] / cltv_c.shape[0]
repeat_rate

0.6706073249884098

In [31]:
#churn rate hesaplama
churn_rate = 1 - repeat_rate
churn_rate

0.3293926750115902

## 5. Profit Margin 
### profit_margin =  total_price * 0.10)

In [34]:
cltv_c["profit_margin"] = cltv_c["total_price"] * 0.10
cltv_c

,total_transaction,total_unit,total_price,average_order_value,purchase_frequence,profit_margin
Customer ID,,,,,,
12346.00000,11,70,372.86000,33.89636,0.00255,37.28600
12347.00000,2,828,1323.32000,661.66000,0.00046,132.33200
12348.00000,1,373,222.16000,222.16000,0.00023,22.21600
12349.00000,3,993,2671.14000,890.38000,0.00070,267.11400
12351.00000,1,261,300.93000,300.93000,0.00023,30.09300
...,...,...,...,...,...,...
18283.00000,6,336,641.77000,106.96167,0.00139,64.17700
18284.00000,1,494,461.68000,461.68000,0.00023,46.16800
18285.00000,1,145,427.00000,427.00000,0.00023,42.70000


## 6.Customer Value 

### customer_value = average_order_value * purchase_frequency

In [40]:
cltv_c["customer_value"] = cltv_c["average_order_value"] * cltv_c["purchase_frequence"]

cltv_c.head()

,total_transaction,total_unit,total_price,average_order_value,purchase_frequence,profit_margin,customer_value
Customer ID,,,,,,,
12346.00000,11,70,372.86000,33.89636,0.00255,37.28600,0.08643
12347.00000,2,828,1323.32000,661.66000,0.00046,132.33200,0.30675
12348.00000,1,373,222.16000,222.16000,0.00023,22.21600,0.05150
12349.00000,3,993,2671.14000,890.38000,0.00070,267.11400,0.61918
12351.00000,1,261,300.93000,300.93000,0.00023,30.09300,0.06976


## 7. Customer Lifetime Value 
### CLTV = (customer_value / churn_rate) x profit_margin

In [44]:
cltv_c["cltv"] = (cltv_c["customer_value"] / churn_rate) * cltv_c["profit_margin"]
cltv_c.head()

,total_transaction,total_unit,total_price,average_order_value,purchase_frequence,profit_margin,customer_value,cltv
Customer ID,,,,,,,,
12346.00000,11,70,372.86000,33.89636,0.00255,37.28600,0.08643,9.78357
12347.00000,2,828,1323.32000,661.66000,0.00046,132.33200,0.30675,123.23546
12348.00000,1,373,222.16000,222.16000,0.00023,22.21600,0.05150,3.47326
12349.00000,3,993,2671.14000,890.38000,0.00070,267.11400,0.61918,502.11041
12351.00000,1,261,300.93000,300.93000,0.00023,30.09300,0.06976,6.37290


## 8. Segmentlerin Oluşturulması

In [47]:
#sıralama
cltv_c.sort_values(by="cltv",ascending=False)

,total_transaction,total_unit,total_price,average_order_value,purchase_frequence,profit_margin,customer_value,cltv
Customer ID,,,,,,,,
18102.00000,89,124216,349164.35000,3923.19494,0.02063,34916.43500,80.93749,8579573.77276
14646.00000,78,170342,248396.50000,3184.57051,0.01808,24839.65000,57.57916,4342070.45829
14156.00000,102,108107,196566.74000,1927.12490,0.02364,19656.67400,45.56484,2719105.08615
14911.00000,205,69722,152147.57000,742.18327,0.04752,15214.75700,35.26833,1629055.80978
13694.00000,94,125893,131443.19000,1398.33181,0.02179,13144.31900,30.46898,1215855.89003
...,...,...,...,...,...,...,...,...
15913.00000,1,3,6.30000,6.30000,0.00023,0.63000,0.00146,0.00279
13788.00000,1,1,3.75000,3.75000,0.00023,0.37500,0.00087,0.00099
14095.00000,1,1,2.95000,2.95000,0.00023,0.29500,0.00068,0.00061


In [49]:
#verilerin incelenmesi
cltv_c.describe().T

,count,mean,std,min,25%,50%,75%,max
total_transaction,4314.00000,4.45410,8.16866,1.00000,1.00000,2.00000,5.00000,205.00000
total_unit,4314.00000,1284.01113,6458.45205,1.00000,158.00000,382.00000,995.25000,220600.00000
total_price,4314.00000,2047.28866,8912.52324,0.00000,307.95000,705.55000,1722.80250,349164.35000
average_order_value,4314.00000,378.14723,492.51721,0.00000,181.95632,286.94600,423.53375,11880.84000
purchase_frequence,4314.00000,0.00103,0.00189,0.00023,0.00023,0.00046,0.00116,0.04752
profit_margin,4314.00000,204.72887,891.25232,0.00000,30.79500,70.55500,172.28025,34916.43500
customer_value,4314.00000,0.47457,2.06595,0.00000,0.07138,0.16355,0.39935,80.93749
cltv,4314.00000,5883.60651,156068.63594,0.00000,6.67369,35.03173,208.87042,8579573.77276


In [54]:
#segmentlere ayırma
cltv_c["segment"] = pd.qcut(cltv_c["cltv"],4,labels = ["D","C","B","A"])

In [57]:
cltv_c.sort_values(by="cltv",ascending=False)

,total_transaction,total_unit,total_price,average_order_value,purchase_frequence,profit_margin,customer_value,cltv,segment
Customer ID,,,,,,,,,
18102.00000,89,124216,349164.35000,3923.19494,0.02063,34916.43500,80.93749,8579573.77276,A
14646.00000,78,170342,248396.50000,3184.57051,0.01808,24839.65000,57.57916,4342070.45829,A
14156.00000,102,108107,196566.74000,1927.12490,0.02364,19656.67400,45.56484,2719105.08615,A
14911.00000,205,69722,152147.57000,742.18327,0.04752,15214.75700,35.26833,1629055.80978,A
13694.00000,94,125893,131443.19000,1398.33181,0.02179,13144.31900,30.46898,1215855.89003,A
...,...,...,...,...,...,...,...,...,...
15913.00000,1,3,6.30000,6.30000,0.00023,0.63000,0.00146,0.00279,D
13788.00000,1,1,3.75000,3.75000,0.00023,0.37500,0.00087,0.00099,D
14095.00000,1,1,2.95000,2.95000,0.00023,0.29500,0.00068,0.00061,D


In [59]:
#segmentlerin analizi

cltv_c.groupby("segment").agg({
    "count","mean","sum"
})

total_transaction                 total_unit                      \
                    count    sum     mean      count      sum       mean   
segment                                                                    
D                    1079   1326  1.22892       1079   117616  109.00463   
C                    1078   2160  2.00371       1078   305135  283.05659   
B                    1078   4063  3.76902       1078   733211  680.15863   
A                    1079  11666 10.81186       1079  4383262 4062.33735   

        total_price                          average_order_value               \
              count           sum       mean               count          sum   
segment                                                                         
D              1079  192265.13000  178.18826                1079 169805.46550   
C              1078  513016.45300  475.89652                1078 317422.35514   
B              1078 1219605.20000 1131.35918                1078 420127.12365   
A              1079 6907116.49100 6401.40546                1079 723972.22347   

                  purchase_frequence                 profit_margin  \
             mean              count     sum    mean         count   
segment                                                              
D       157.37300               1079 0.30737 0.00028          1079   
C       294.45487               1078 0.50070 0.00046          1078   
B       389.72832               1078 0.94182 0.00087          1078   
A       670.96592               1079 2.70422 0.00251          1079   

                               customer_value                     cltv  \
                 sum      mean          count        sum    mean count   
segment                                                                  
D        19226.51300  17.81883           1079   44.56772 0.04130  1079   
C        51301.64530  47.58965           1078  118.91897 0.11031  1078   
B       121960.52000 113.13592           1078  282.70867 0.26225  1078   
A       690711.64910 640.14055           1079 1601.09330 1.48387  1079   

                                    
                   sum        mean  
segment                             
D           2849.32790     2.64071  
C          18184.26342    16.86852  
B         103549.86329    96.05739  
A       25257295.04118 23408.05843

In [74]:
#cltv'e aktarmak
cltv_c.to_csv("cltv_c.csv")

---------

In [80]:
df

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,total_price
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95000,13085.00000,United Kingdom,83.40000
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75000,13085.00000,United Kingdom,81.00000
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75000,13085.00000,United Kingdom,81.00000
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10000,13085.00000,United Kingdom,100.80000
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25000,13085.00000,United Kingdom,30.00000
...,...,...,...,...,...,...,...,...,...
525456,538171,22271,FELTCRAFT DOLL ROSIE,2,2010-12-09 20:01:00,2.95000,17530.00000,United Kingdom,5.90000
525457,538171,22750,FELTCRAFT PRINCESS LOLA DOLL,1,2010-12-09 20:01:00,3.75000,17530.00000,United Kingdom,3.75000
525458,538171,22751,FELTCRAFT PRINCESS OLIVIA DOLL,1,2010-12-09 20:01:00,3.75000,17530.00000,United Kingdom,3.75000
525459,538171,20970,PINK FLORAL FELTCRAFT SHOULDER BAG,2,2010-12-09 20:01:00,3.75000,17530.00000,United Kingdom,7.50000


### 9. BONUS: Tüm İşlemlerin Fonksiyonlaştırılması

In [139]:
import datetime as dt
import pandas as pd
def create_cltv(df,csv=False,profit=0.10):
    #verilerin hazırlanması
    df = df[~df["Invoice"].str.contains("C", na=False)]
    df = df[(df["Quantity"]>0)]
    df.dropna(inplace=True)
    df["Total_Price"] = df["Quantity"] * df["Price"]
    
    #gerekli değişkenlerin oluşturulması
    cltv_c = df.groupby("Customer ID").agg({
        "Invoice" : lambda x:x.nunique(), #total_transaction
        "Quantity" : lambda x:x.sum(), #total unit
        "Total_Price" : lambda x:x.sum() #total price
    })
    cltv_c.columns=["total_transaction","tolal_unit","total_price"]
    
    #average_order_value
    cltv_c["average_order_value"] = cltv_c["total_price"] / cltv_c["total_transaction"]
    
    #purchase_frequence
    cltv_c["purchase_frequence"] = cltv_c["total_transaction"] / cltv_c.shape[0]
    
    #repeat_rate
    repeat_rate = cltv_c[cltv_c["total_transaction"]>1].shape[0] / cltv_c.shape[0]
    
    #churn_rate
    churn_rate = 1 - repeat_rate
    
    #customer_value
    cltv_c["customer_value"] = cltv_c["average_order_value"] * cltv_c["purchase_frequence"]
    
    #profit_margin
    cltv_c["profit_margin"] = cltv_c["total_price"]*profit
    
    #cltv
    cltv_c["cltv"] = (cltv_c["customer_value"]/churn_rate)*cltv_c["profit_margin"]
    
    #segment
    cltv_c["segment"] = pd.qcut(cltv_c["cltv"],4,labels=["D","C","B","A"])
    
    ay = dt.datetime.now().month
    if csv:
        cltv_c.to_csv(f"{ay}._cltv_raporu.csv")
    return cltv_c

In [140]:
df = df_.copy()

In [141]:
create_cltv(df)

,total_transaction,tolal_unit,total_price,average_order_value,purchase_frequence,customer_value,profit_margin,cltv,segment
Customer ID,,,,,,,,,
12346.00000,11,70,372.86000,33.89636,0.00255,0.08643,37.28600,9.78357,C
12347.00000,2,828,1323.32000,661.66000,0.00046,0.30675,132.33200,123.23546,B
12348.00000,1,373,222.16000,222.16000,0.00023,0.05150,22.21600,3.47326,D
12349.00000,3,993,2671.14000,890.38000,0.00070,0.61918,267.11400,502.11041,A
12351.00000,1,261,300.93000,300.93000,0.00023,0.06976,30.09300,6.37290,D
...,...,...,...,...,...,...,...,...,...
18283.00000,6,336,641.77000,106.96167,0.00139,0.14876,64.17700,28.98443,C
18284.00000,1,494,461.68000,461.68000,0.00023,0.10702,46.16800,14.99989,C
18285.00000,1,145,427.00000,427.00000,0.00023,0.09898,42.70000,12.83103,C


In [142]:
a = create_cltv(df)

In [143]:
a.cltv

Customer ID
12346.00000     9.78357
12347.00000   123.23546
12348.00000     3.47326
12349.00000   502.11041
12351.00000     6.37290
                 ...   
18283.00000    28.98443
18284.00000    14.99989
18285.00000    12.83103
18286.00000   118.27803
18287.00000   387.21713
Name: cltv, Length: 4314, dtype: float64